In [4]:
import numpy as np
from numpy import array, arange, zeros, ones
import pickle

In [5]:
n = 7
prob_list = np.round(arange(0.3, 1.0, 0.1), 2)
p_min = 1
p_max = 70
s_min = 0
s_max = 49
r_min = 0
r_max = 19

In [8]:
path = "/home/kardashin/Work/QAOA-QFI/Data/Raw/CUT/Random_graphs/"
pref = "maxcut-random_graphs-"
post = "-prevsol" 

In [9]:
fvals_list = []
xfs_list = []
funs_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1, p_max - p_min + 1])
errors_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1, p_max - p_min + 1])
nits_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1, p_max - p_min + 1])
nfevs_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1, p_max - p_min + 1])
times_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1, p_max - p_min + 1])
EQD_list = zeros([len(prob_list), s_max - s_min + 1, r_max - r_min + 1])
H_diag_list = zeros([len(prob_list), s_max - s_min + 1, 2**n])

for prob_i in range(len(prob_list)):

    folder = 'n=%d-prob=%.2f-p=(%d,%d)-prevsol/' %(n, prob_list[prob_i], p_min, p_max)

    fvals_m = []
    xfs_m = []
    for s in range(s_min, s_max + 1):
        fvals_s = []
        xfs_s = []
        for r in range(r_min, r_max + 1):
            print("prob=%.2f | s=%d | r=%d" %(prob_list[prob_i], s, r), end="\r")
            try:
                file_name = path + folder + pref + "n=%d-prob=%.2f-p=(%d,%d)-s=%d-r=%d" %(n, prob_list[prob_i], p_min, p_max, s, r) + post + ".npy"
                _, _, _, _, _, _, _, H_diag_r, fvals_r, funs_r, errors_r, nits_r, nfevs_r, times_r, xfs_r, EQD_r = np.load(file_name, allow_pickle=True)
                fvals_s.append(fvals_r)
                xfs_s.append(xfs_r)
                funs_list[prob_i][s][r] = funs_r
                errors_list[prob_i][s][r] = errors_r
                nits_list[prob_i][s][r] = nits_r
                nfevs_list[prob_i][s][r] = nfevs_r
                times_list[prob_i][s][r] = times_r
                EQD_list[prob_i][s][r] = EQD_r
            except:
                print("\tNo data for (m=%d, s=%d, r=%d)" %(prob_list[prob_i], s, r))
                fvals_s.append(100*ones(1))
                xfs_s.append(zeros(2*p_max))
                funs_list[prob_i][s][r] = 100*ones(p_max - p_min + 1)
                errors_list[prob_i][s][r] = 100*ones(p_max - p_min + 1)
                nits_list[prob_i][s][r] = zeros(p_max - p_min + 1)
                nfevs_list[prob_i][s][r] = zeros(p_max - p_min + 1)
                times_list[prob_i][s][r] = zeros(p_max - p_min + 1)
                EQD_list[prob_i][s][r] = 2**n
    
        fvals_m.append(fvals_s)
        xfs_m.append(xfs_s)
        H_diag_list[prob_i][s] = H_diag_r
    
    fvals_list.append(fvals_m)
    xfs_list.append(xfs_m)

p_c_list = np.floor(np.max(EQD_list, axis=2)/2)

prob=0.90 | s=49 | r=19

In [10]:
path = "/home/kardashin/Work/QAOA-QFI/Data/"
file_name = path + "cut-" + f"n={n}-prob=({prob_list[0]},{prob_list[-1]})-p=({p_min},{p_max})-s=({s_min},{s_max})-r=({r_min},{r_max})" + post 

data_dict = {
    "n": n,
    "prob_list": prob_list,
    "s_min": s_min,
    "s_max": s_max,
    "r_min": r_min,
    "r_max": r_max,
    "p_min": p_min,
    "p_max": p_max,
    "funs_list": funs_list,
    "errors_list": errors_list,
    "nits_list": nits_list,
    "nfevs_list": nfevs_list,
    "times_list": times_list,
    "EQD_list": EQD_list,
    "H_diag_list": H_diag_list,
    # "fvals_list": fvals_list,
    # "xfs_list": xfs_list,
    "note": "p -> p+1 initialization (prevsol). BFGS optimizer", 
} 

with open(file_name + ".dict", 'wb') as fp:
    pickle.dump(data_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)